# Tool Call LLMs
Some of the more exciting use cases for RT is when you want to cede control of the decision making process to the LLM. RT has a suite of tools that you can use to make this process much simpler. Check out an example below. 

In [1]:
from typing import Dict, Any, Set, Type
import railtracks as rt

In [2]:
# Common params for all terminal_llms being used as tools in this example
critic_tool_params = {
    rt.llm.Parameter("analysis_detail", "string", "The thing you would like to analyze")
}

critic_model = rt.llm.OpenAILLM("gpt-4o")

In [3]:
HarshCritic = rt.library.terminal_llm(
    pretty_name="Harsh Critic",
    system_message=rt.llm.SystemMessage(
        "You are a harsh critic of the world and you should analyze the given statement and provide a harsh critique of it. Be very concise and to the point."
    ),
    llm_model=critic_model,
    tool_details="A tool used to critique a statement harshly.",
    tool_params=critic_tool_params,
)

PositiveCritic = rt.library.terminal_llm(
    pretty_name="Positive Critic",
    system_message=rt.llm.SystemMessage(
        "You are a positive critic of the world and you should analyze the given statement and provide a positive critique of it. Be very concise and to the point."
    ),
    llm_model=critic_model,
    tool_details="A tool used to critique a statement positively.",
    tool_params=critic_tool_params,
)

DeeperMeaningCritic = rt.library.terminal_llm(
    pretty_name="Deeper Meaning Critic",
    system_message=rt.llm.SystemMessage(
        "You are a critic of the world and you should analyze the given statement and provide a critique of it. Be very concise and to the point."
    ),
    llm_model=critic_model,
    tool_details="A tool used to critique a statement deeply.",
    tool_params=critic_tool_params,
)

In [4]:
system_message_critic = rt.llm.SystemMessage(
    "You are a critic of the world and you provide comprehensive critiques of the world around you. You should utilize the provided tools to collect specific critiques and structure them before completing your answer."
)
Critic = rt.library.message_hist_tool_call_llm(
    connected_nodes={HarshCritic, PositiveCritic, DeeperMeaningCritic},
    pretty_name="Critic",
    system_message=system_message_critic,
    llm_model=rt.llm.OpenAILLM("gpt-4o"),
)

C:\Users\Aryan\dev\rc\src\railtracks\exceptions\node_creation\validation.py:219: UserWarning: You have set max_tool_calls to None. The llm will be able to make unlimited tool calls.
  warnings.warn(


In [5]:
template = (
    "I am writing a short story and I would like to analyze my introduction.\n"
    "\n"
    "Once upon a time there was a little boy who lived in a small village. He was a very kind and generous, but lacked an understanding"
    " of the world around him. He was always looking for ways to help others and make the world a better place. One day, he stumbled upon a"
    " magical book that would change his life forever. The book was filled with stories of adventure and mystery, and the promise of a better"
    " tomorrow."
)
with rt.Runner(
    executor_config=rt.run.ExecutorConfig(logging_setting="QUIET", timeout=50)
) as runner:
    response = await runner.run(
        Critic, user_input=rt.llm.MessageHistory([rt.llm.UserMessage(template)])
    )

C:\Users\Aryan\dev\rc\src\railtracks\exceptions\node_invocation\validation.py:46: RuntimeWarning: You have set max_tool_calls to None. The llm will be able to make unlimited tool calls.
  warnings.warn(


In [6]:
response.answer

[system: You are a critic of the world and you provide comprehensive critiques of the world around you. You should utilize the provided tools to collect specific critiques and structure them before completing your answer.,
 user: I am writing a short story and I would like to analyze my introduction.
 
 Once upon a time there was a little boy who lived in a small village. He was a very kind and generous, but lacked an understanding of the world around him. He was always looking for ways to help others and make the world a better place. One day, he stumbled upon a magical book that would change his life forever. The book was filled with stories of adventure and mystery, and the promise of a better tomorrow.,
 assistant: [ToolCall(identifier='call_bVXGfrdcygJ76IILaQvI7wSt', name='Harsh_Critic', arguments={'analysis_detail': 'Once upon a time there was a little boy who lived in a small village. He was a very kind and generous, but lacked an understanding of the world around him. He was al

Sometimes there may be a need to inject parameters into the subserviant tools at instance "runtime". This can be done by modifying the node creation method in the parent top level node


In [7]:
class GradeLevelHarshCritic(HarshCritic):
    def __init__(self, message_history: rt.llm.MessageHistory, grade_level: int):
        super().__init__(message_history=message_history)
        self.grade_level = grade_level

    @classmethod
    def prepare_tool(cls, tool_parameters):
        message_hist = rt.llm.MessageHistory(
            [rt.llm.UserMessage(tool_parameters["analysis_detail"])]
        )
        return cls(message_hist, tool_parameters["grade_level"])

    def system_message(self):
        return (
            super().system_message()
            + " You should provide a critique at a grade level of "
            + str(self.grade_level)
        )

In [8]:
class InjectGradeLevel(Critic):
    def __init__(self, message_history: rt.llm.MessageHistory, grade_level: int):
        super().__init__(message_history=message_history)
        self.grade_level = grade_level

    def create_node(self, tool_name: str, arguments: Dict[str, Any]) -> rt.Node:
        if tool_name == "Harsh_Critic":
            arguments["grade_level"] = self.grade_level
            return GradeLevelHarshCritic.prepare_tool(arguments)
        # note the super method will not add any parameters and work just as it always does.
        return super().create_node(tool_name, arguments)

    def connected_nodes(self) -> Set[Type[rt.Node]]:
        # note that you can add the new node to the connected nodes set
        return {GradeLevelHarshCritic, PositiveCritic, DeeperMeaningCritic}

In [9]:
template = (
    "I am writing a short story and I would like to analyze my introduction.\n"
    "\n"
    "Green Apples are a strong fruit that can be used in many different ways. They are a great source of vitamins and minerals, and can be eaten or even thrown at others you don't like"
)
# with rt.Runner(executor_config=rt.run.ExecutorConfig(logging_setting="QUIET", timeout=50)) as runner:
#     response = await runner.run(InjectGradeLevel,
#         message_history=rt.llm.MessageHistory([rt.llm.UserMessage(template)]),
#         grade_level=12)

rt.call(
    InjectGradeLevel,
    message_history=rt.llm.MessageHistory([rt.llm.UserMessage(template)]),
    grade_level=12,
)


print(response.answer)

C:\Users\Aryan\AppData\Local\Temp\ipykernel_41084\2308724356.py:11: RuntimeWarning: coroutine 'call' was never awaited
  rt.call(


system: You are a critic of the world and you provide comprehensive critiques of the world around you. You should utilize the provided tools to collect specific critiques and structure them before completing your answer.
user: I am writing a short story and I would like to analyze my introduction.

Once upon a time there was a little boy who lived in a small village. He was a very kind and generous, but lacked an understanding of the world around him. He was always looking for ways to help others and make the world a better place. One day, he stumbled upon a magical book that would change his life forever. The book was filled with stories of adventure and mystery, and the promise of a better tomorrow.
assistant: [ToolCall(identifier='call_bVXGfrdcygJ76IILaQvI7wSt', name='Harsh_Critic', arguments={'analysis_detail': 'Once upon a time there was a little boy who lived in a small village. He was a very kind and generous, but lacked an understanding of the world around him. He was always lo